# import libraries

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime

# Set the maximum number of displayed columns to None, which means display all columns
pd.set_option('display.max_columns', None)


# Load the Dataset

In [2]:
# demand data loading
demand_data = pd.read_excel('./data/All Demands_Data Dump.xlsx', engine='openpyxl')

In [3]:
demand_data.shape

(14849, 69)

In [4]:
# mapping data loading
map_data = pd.read_excel('./data/Mapping.xlsx', engine='openpyxl')

In [5]:
map_data

,BUOps,BU
0,BU2-Suganthi,ASHISH
1,BU1-Muthu,ASIF
2,BU1-Muthu,GLOBAL
3,BU1-Muthu,KISHOR
4,BU1-Muthu,KISHOR - OTHERS
5,FL1-Preethi,MARS
6,FL2-Madhav,PEPSI
7,BU1-Muthu,PRASHANT
8,BU2-Suganthi,RACHIT
9,BU1-Muthu,SETHU - OTHERS


In [6]:
# saving the different mappings for BU in a dictionary
map_dictionary = {}
for name in map_data['BU'].unique():
    map_dictionary[name] = map_data[map_data.BU==name]['BUOps'].iloc[0]


In [7]:
map_dictionary

{'ASHISH': 'BU2-Suganthi',
 'ASIF': 'BU1-Muthu',
 'GLOBAL': 'BU1-Muthu',
 'KISHOR': 'BU1-Muthu',
 'KISHOR - OTHERS': 'BU1-Muthu',
 'MARS': 'FL1-Preethi',
 'PEPSI': 'FL2-Madhav',
 'PRASHANT': 'BU1-Muthu',
 'RACHIT': 'BU2-Suganthi',
 'SETHU - OTHERS': 'BU1-Muthu',
 'US OTHERS': 'BU1-Muthu'}

## creating Status date, Demand from date, Ops Plan date, BSD date

In [8]:
# dates are in string format so converting this into datetime format
demand_data['Status Date'] = pd.to_datetime(demand_data['Status Date'], errors = 'coerce')
demand_data['Demand From']= pd.to_datetime(demand_data['Demand From'], errors = 'coerce')
demand_data['Ops Planned Date']= pd.to_datetime(demand_data['Ops Planned Date'], errors = 'coerce')
demand_data['Billability Start Date']= pd.to_datetime(demand_data['Billability Start Date'], errors = 'coerce')

C:\Users\prakash.pandey\AppData\Local\Temp\ipykernel_14572\4213420611.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  demand_data['Status Date'] = pd.to_datetime(demand_data['Status Date'], errors = 'coerce')


In [9]:
# creating month column
demand_data['Status_month'] = demand_data['Status Date'].dt.strftime('%B %Y')
demand_data['Demand_from_month'] = demand_data['Demand From'].dt.strftime('%B %Y')
demand_data['Ops_date_month']= demand_data['Ops Planned Date'].dt.strftime('%B %Y')
demand_data['B_start_month'] = demand_data['Billability Start Date'].dt.strftime('%B %Y')

## Mappling BU and BUOps

In [10]:
# creating a new column "BUOps" and mapping values from BU column
demand_data['BUOps'] = None
for val in map_dictionary:
    demand_data.loc[(demand_data.BU==val), 'BUOps'] = map_dictionary[val]

## applying filters

In [11]:
#demand type = new
demand_data = demand_data[demand_data['Demand Type']=='New']

# Allocation Sub Type = Billable only
demand_data = demand_data[demand_data['Allocation Sub Type']=='Billable']

# removing internal demands that starts with RE means those has been extended
demand_data = demand_data[~demand_data['Demand No.'].str.startswith('RE')]

# removing cancelled or hold status demand
demand_data = demand_data[~demand_data['Status'].isin(['Canceled', 'Hold', 'Open'])]

# filter on ramp up reasons
demand_data = demand_data[~demand_data['Ramp Up Reason'].isin(['P2P', "SOW Extension", 'Grace Extension (awaiting SOW Renewal)', 'Billable to Investment/NB'])]

# filter on ramp down reason
demand_data = demand_data[~demand_data['Ramp Down Reason'].isin(['InCorrect Demand'])]

In [12]:
demand_data['Ramp Up Reason'].value_counts(dropna=False)

B2P                              885
NaN                              637
AB2P                             293
Investment/NB to Billable        255
A2A                              187
Project Extension (SOW ended)      1
Name: Ramp Up Reason, dtype: int64

In [13]:
# just checking the minimum Ops planned date in the data
demand_data[demand_data['Ops Planned Date']==demand_data['Ops Planned Date'].min()]

,S.No.,Requested Date,Requested By,Demand No.,Created By,Customer Reporting Geo,Customer No.,Customer Name,Customer Code,BU,Customer Priority,Project No.,Project Name,Project Type,Thread No.,Customer Acceptance,Demand Type,Status,Status Date,Demand From,Demand To,Ops Planned Date,Sub Business Unit,Stream,Sub Stream,Track,Level,Tigerway Role,Fulfillment Channel,Is Billable,Allocation Sub Type,Allocation From Date,Allocation To Date,Billability Start Date,Bill Rate,Demand Classification,Delivery Partner,Client Partner,Tech Partner,Engineering Partner,Is Seed?,Is Corrective Request?,Shortlisted Resources,Current Allocation From,Current Allocation To,Ramp Up Reason,Ramp Down Reason,Hold Date,Hold Reason,Cancel Date,Cancel Reason,Allocated Resource,Allocated Resource - Employee ID,Primary Skills,Secondary Skills,Feb 23 Allocation Hours,Feb 23 Allocation %,Mar 23 Allocation Hours,Mar 23 Allocation %,Apr 23 Allocation Hours,Apr 23 Allocation %,May 23 Allocation Hours,May 23 Allocation %,Jun 23 Allocation Hours,Jun 23 Allocation %,Jul 23 Allocation Hours,Jul 23 Allocation %,Aug 23 Allocation Hours,Aug 23 Allocation %,Status_month,Demand_from_month,Ops_date_month,B_start_month,BUOps
2917,2918,2023-06-28,NaN,R-0012590-01,Madhava Krishnan C V,US,C00066,PEPSICO,PEPS,PEPSI,P1,DP00856,PEPSICO_DATA FOUNDATION_EMEA,Team Extension - T&M,DFL-00856-01,No,New,Initiate External Hiring,2023-11-08,2023-07-17,2023-12-29,2000-01-01,Technology,Engineering,Data Warehouse,NaN,A2,IC,Contract,Yes,Billable,2023-07-17,2023-12-29,2023-07-17,32,LL,Pavan Kumar Sristi,Karthik Balakrishnan Gopalakrishnan,NaN,Manish Gupta,No,No,NaN,2023-07-17 00:00:00,2023-12-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAP BW,NaN,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,88.0,100.00%,184.0,100.00%,November 2023,July 2023,January 2000,July 2023,FL2-Madhav
3528,3529,2023-07-18,NaN,R-0013154-01,Madhava Krishnan C V,US,C00066,PEPSICO,PEPS,PEPSI,P1,DP00856,PEPSICO_DATA FOUNDATION_EMEA,Team Extension - T&M,DFL-00856-01,No,New,Initiate External Hiring,2023-08-18,2023-08-01,2023-12-29,2000-01-01,Technology,Engineering,Data Warehouse,NaN,A2,IC,Contract,Yes,Billable,2023-08-01,2023-12-29,2023-08-01,NaN,Non Mainstream-Standard-LL,Pavan Kumar Sristi,Karthik Balakrishnan Gopalakrishnan,NaN,Manish Gupta,No,No,NaN,2023-08-01 00:00:00,2023-12-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAP BW,NaN,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,184.0,100.00%,August 2023,August 2023,January 2000,August 2023,FL2-Madhav
3529,3530,2023-07-18,NaN,R-0013154-02,Madhava Krishnan C V,US,C00066,PEPSICO,PEPS,PEPSI,P1,DP00856,PEPSICO_DATA FOUNDATION_EMEA,Team Extension - T&M,DFL-00856-01,No,New,Initiate External Hiring,2023-08-18,2023-08-01,2023-12-29,2000-01-01,Technology,Engineering,Data Warehouse,NaN,A2,IC,Contract,Yes,Billable,2023-08-01,2023-12-29,2023-08-01,NaN,Non Mainstream-Standard-LL,Pavan Kumar Sristi,Karthik Balakrishnan Gopalakrishnan,NaN,Manish Gupta,No,No,NaN,2023-08-01 00:00:00,2023-12-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SAP BW,NaN,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,184.0,100.00%,August 2023,August 2023,January 2000,August 2023,FL2-Madhav
4173,4174,2023-08-04,NaN,R-0013788-01,Sreepriya Subbaraman,US,C00066,PEPSICO,PEPS,PEPSI,P1,DP00975,PEPSICO_PEPWISE V&V OBSERVABILITY,Managed Program - FP,DFL-00975-01,No,New,Initiate External Hiring,2023-08-17,2023-08-18,2023-12-29,2000-01-01,Technology,Engineering,Business Intelligence,NaN,A2,IC,Contract,Yes,Billable,2023-08-18,2023-12-29,2023-08-18,30,Mainstream-Standard-HL,Pavan Kumar Sristi,Sethu Janardhanan,NaN,Manish Gupta,No,No,NaN,2023-08-18 00:00:00,2023-12-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,0.0,0.00%,80.0,100.00%,August 2023,August 2023,January 2000,August 2023,FL2-Madhav
4617,4618,2022-03-22,NaN,R-2793-01,Ananya Biswas^,US,C00155,JOHNSONBROTHERS,JB11,SETHU - OTHERS,P4,DP00304,JOHNSONBROTHERS,COE - Monthly Retainer,DFL-00304-01,No,

In [14]:
# demand_data1 = pd.read_excel('All Demands_Data Dump.xlsx', engine='openpyxl')
demand_data[demand_data['Ops Planned Date']==demand_data['Ops Planned Date'].min()][['Demand Type', 'Allocation Sub Type',
                                                                                     'Demand No.', 'Status', 'Ramp Up Reason',
                                                                                     'Ramp Down Reason', 'Ops Planned Date']]

,Demand Type,Allocation Sub Type,Demand No.,Status,Ramp Up Reason,Ramp Down Reason,Ops Planned Date
2917,New,Billable,R-0012590-01,Initiate External Hiring,NaN,NaN,2000-01-01
3528,New,Billable,R-0013154-01,Initiate External Hiring,NaN,NaN,2000-01-01
3529,New,Billable,R-0013154-02,Initiate External Hiring,NaN,NaN,2000-01-01
4173,New,Billable,R-0013788-01,Initiate External Hiring,NaN,NaN,2000-01-01
4617,New,Billable,R-2793-01,Fulfilled,NaN,NaN,2000-01-01


In [15]:
# demand_data1 = pd.read_excel('All Demands_Data Dump.xlsx', engine='openpyxl')
demand_data[demand_data['Ops Planned Date']==demand_data['Ops Planned Date'].min()]['Ramp Down Reason'].value_counts(dropna=False)

NaN    5
Name: Ramp Down Reason, dtype: int64

In [16]:
# demand_data1 = pd.read_excel('All Demands_Data Dump.xlsx', engine='openpyxl')
demand_data[demand_data['Ops Planned Date']==demand_data['Ops Planned Date'].min()]['Ramp Up Reason'].value_counts(dropna=False)

NaN    5
Name: Ramp Up Reason, dtype: int64

# enter the input month

In [17]:
# please change the month and year according to the dataset
month = "August 2023"

## Reporting status column

In [19]:
# Aug fullfiled : Status = Fulfiled & Status month = Aug 23 (regardless of BSD month)
# D Univ : Status != Fulfiled
# Prior Fulfilled, Aug BSD : Status = Fulfiled & BSD month = Aug 23
# what about : Prior Fulfilled, BSD not in Aug, May be in sep, or october
month_fulfilled =  f'{month[:3]} fulfilled'
demand_data['Reporting_status'] = None
demand_data.loc[(demand_data.Status=='Fulfilled') & (demand_data.Status_month==month), 'Reporting_status'] = month_fulfilled
demand_data.loc[(demand_data.Status=='Fulfilled') & (demand_data.Status_month!=month) & (demand_data.B_start_month==month), 'Reporting_status'] = f'Prior Fulfilled, {month[:3]} BSD'
demand_data.loc[(demand_data.Status!='Fulfilled'), 'Reporting_status'] = 'D Universe'

In [21]:
demand_data.Reporting_status.value_counts()

D Universe                  165
Prior Fulfilled, Aug BSD    157
Aug fulfilled                84
Name: Reporting_status, dtype: int64

In [20]:
demand_data.Reporting_status.value_counts()

D Universe                  165
Prior Fulfilled, Aug BSD    157
Aug fulfilled                84
Name: Reporting_status, dtype: int64

In [22]:
# billing start month before input month (ex:aug), status date before aug, delete :  
filter_date = datetime.strptime(month, '%B %Y')
demand_data['B_start_month'] = pd.to_datetime(demand_data['B_start_month'], format='%B %Y')
demand_data['Status_month'] = pd.to_datetime(demand_data['Status_month'], format='%B %Y')
demand_data = demand_data[~((demand_data.B_start_month<filter_date) & (demand_data.Status_month<filter_date) & (demand_data.Status=="Fulfilled"))]

In [23]:
demand_data['Status_month'] = demand_data['Status Date'].dt.strftime('%B %Y')
demand_data['B_start_month'] = demand_data['B_start_month'].dt.strftime('%B %Y')

In [25]:
demand_data.shape

(599, 75)

In [26]:
demand_data.Reporting_status.value_counts(dropna=False)

None                        209
Prior Fulfilled, Aug BSD    157
D Universe                  149
Aug fulfilled                84
Name: Reporting_status, dtype: int64

In [27]:
demand_data.Status.value_counts()

Fulfilled                   450
In Identification           129
Mapped                       10
Initiate External Hiring      7
Finalized                     3
Name: Status, dtype: int64

In [28]:
demand_data.shape

(599, 75)

## Stream1 column

In [29]:
demand_data['Sub Stream'].value_counts()

Big Data Engineering       115
Data Science                87
Data Warehouse              72
Application Engineering     59
Business Intelligence       36
MLE                         35
Quality Engineering         29
Devops                      15
Data Science Python          8
MLOPS                        8
Data Science Insights        4
Name: Sub Stream, dtype: int64

In [30]:
demand_data.Stream.value_counts(dropna=False)

Engineering              326
Data Science             147
Analytics Consulting      75
Technology Consulting     31
Design Consulting         14
Production Support         6
Name: Stream, dtype: int64

In [31]:
# creating a new column Stream1 from stream and sub-stream
demand_data['Stream1'] = None
demand_data.loc[(demand_data['Sub Stream']=='Big Data Engineering'), 'Stream1'] = 'DE'
demand_data.loc[(demand_data['Sub Stream']=='Data Warehouse'), 'Stream1'] = 'DE'
demand_data.loc[(demand_data['Sub Stream']=='Quality Engineering'), 'Stream1'] = 'DE'
demand_data.loc[(demand_data['Sub Stream']=='Business Intelligence'), 'Stream1'] = 'DE'
demand_data.loc[(demand_data['Sub Stream']=='Data Science'), 'Stream1'] = 'DS'
demand_data.loc[(demand_data['Sub Stream']=='Application Engineering'), 'Stream1'] = 'APP & Design'
demand_data.loc[(demand_data['Sub Stream']=='MLE'), 'Stream1'] = 'MLE'
demand_data.loc[(demand_data['Sub Stream']=='Devops'), 'Stream1'] = 'DevOps'
demand_data.loc[(demand_data['Sub Stream']=='Data Science Python'), 'Stream1'] = 'DS'
demand_data.loc[(demand_data['Sub Stream']=='MLOPS'), 'Stream1'] = 'MLE'
demand_data.loc[(demand_data['Sub Stream']=='Data Science Insights'), 'Stream1'] = 'DS'

demand_data.loc[(demand_data['Sub Stream'].isna()) & (demand_data.Stream=='Analytics Consulting'), 'Stream1'] = 'AC'
demand_data.loc[(demand_data['Sub Stream'].isna()) & (demand_data.Stream=='Technology Consulting'), 'Stream1'] = 'TC'
demand_data.loc[(demand_data['Sub Stream'].isna()) & (demand_data.Stream=='Design Consulting'), 'Stream1'] = 'APP & Design'
demand_data.loc[(demand_data['Sub Stream'].isna()) & (demand_data.Stream=='Production Support'), 'Stream1'] = 'Prod Support'
demand_data.loc[(demand_data['Sub Stream'].isna()) & (demand_data.Stream=='Data Science'), 'Stream1'] = 'DS'


In [32]:
demand_data.Stream1.value_counts(dropna=False)

DE              252
DS              104
AC               75
APP & Design     73
MLE              43
TC               31
DevOps           15
Prod Support      6
Name: Stream1, dtype: int64

# FTE 

In [33]:
# storing all the months related column in a list
allocation_months = [col for col in demand_data.columns if 'Allocation %' in col]

#reversing the order of the months
allocation_months = allocation_months[::-1]

In [37]:
for col in allocation_months:
    demand_data[col] = demand_data[col].str.rstrip('%').astype(float)

In [38]:
# creating a new column name FTE with some conditions related to months column
demand_data['FTE'] = 0
for col in allocation_months:
    demand_data.loc[(demand_data.FTE==0), 'FTE'] = demand_data[col]/100


In [39]:
demand_data.FTE.value_counts()

1.0000    546
0.5000     33
0.2500      9
0.4000      2
0.8000      2
0.1500      2
0.7038      1
0.2000      1
0.1000      1
0.7500      1
0.3000      1
Name: FTE, dtype: int64

In [33]:
# demand_data.to_excel('demand_after_filters.xlsx')

In [40]:
demand_data.head()

,S.No.,Requested Date,Requested By,Demand No.,Created By,Customer Reporting Geo,Customer No.,Customer Name,Customer Code,BU,Customer Priority,Project No.,Project Name,Project Type,Thread No.,Customer Acceptance,Demand Type,Status,Status Date,Demand From,Demand To,Ops Planned Date,Sub Business Unit,Stream,Sub Stream,Track,Level,Tigerway Role,Fulfillment Channel,Is Billable,Allocation Sub Type,Allocation From Date,Allocation To Date,Billability Start Date,Bill Rate,Demand Classification,Delivery Partner,Client Partner,Tech Partner,Engineering Partner,Is Seed?,Is Corrective Request?,Shortlisted Resources,Current Allocation From,Current Allocation To,Ramp Up Reason,Ramp Down Reason,Hold Date,Hold Reason,Cancel Date,Cancel Reason,Allocated Resource,Allocated Resource - Employee ID,Primary Skills,Secondary Skills,Feb 23 Allocation Hours,Feb 23 Allocation %,Mar 23 Allocation Hours,Mar 23 Allocation %,Apr 23 Allocation Hours,Apr 23 Allocation %,May 23 Allocation Hours,May 23 Allocation %,Jun 23 Allocation Hours,Jun 23 Allocation %,Jul 23 Allocation Hours,Jul 23 Allocation %,Aug 23 Allocation Hours,Aug 23 Allocation %,Status_month,Demand_from_month,Ops_date_month,B_start_month,BUOps,Reporting_status,Stream1,FTE
15,16,2023-03-16,NaN,R-0010012-01,Arunprasad A,US,C00066,PEPSICO,PEPS,PEPSI,P1,DP00528,PEPSICO_DATA FOUNDATION_FP&A EUROPE,Team Extension - T&M,DFL-00528-01,No,New,Fulfilled,2023-11-05,2023-05-11,2023-06-30,2023-05-25,Technology,Engineering,Big Data Engineering,BDE AZURE,L1,IC,Internal,Yes,Billable,2023-05-11,2023-06-30,2023-05-11,32,ML,Pavan Kumar Sristi,Karthik Balakrishnan Gopalakrishnan,NaN,Manish Gupta,No,No,NaN,2023-05-11 00:00:00,2023-06-30 00:00:00,B2P,Exit - With Replacement in Current Month,NaN,NaN,NaN,NaN,Mohammad Younus Shaik^,5229,PySpark,Azure Databricks,0.0,0.0,0.0,0.0,0.0,0.0,120.0,100.0,176.0,100.0,0.0,0.0,0.0,0.0,November 2023,May 2023,May 2023,May 2023,FL2-Madhav,None,DE,1.0
18,19,2023-03-16,NaN,R-0010015-01,Preethi Vinnarasi Lourdhu nathan,US,C00048,MARS,MARS,MARS,P1,DP00455,MARS_MEA_MW_ODD_PHASE 2_2023,Managed Program - FP,DFL-00455-01,No,New,Fulfilled,2023-10-04,2023-04-10,2023-04-30,2023-04-15,Technology,Engineering,Quality Engineering,Quality Engineering - APP,A1,IC,Internal,Yes,Billable,2023-04-10,2023-04-30,2023-04-11,35,HL,Rahul Jain,Shachin Prabhat,Aswani Kumar Yeraguntla,Manish Gupta,No,No,NaN,2023-04-10 00:00:00,2023-04-30 00:00:00,A2A,NaN,NaN,NaN,NaN,NaN,Amar Namdev Memane,2640,Automation Testing,Python,0.0,0.0,0.0,0.0,120.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,October 2023,April 2023,April 2023,April 2023,FL1-Preethi,None,DE,1.0
167,168,2023-03-22,NaN,R-0010146-01,Allen Valentina Jayasagaran,US,C00097,ZEBRA,ZTC1,ASHISH,P4,DP00738,ZEBRA_SERVICE DATA CENTRALIZATION,Managed Program - FP,DFL-00738-01,No,New,Fulfilled,2023-10-04,2023-04-15,2023-09-22,2023-04-15,Technology,Engineering,Big Data Engineering,BDE AWS,L1,IC,Internal,Yes,Billable,2023-04-15,2023-09-22,2023-04-17,35,ML,Tarun Goel,Kunal Jain,Tirthankar Chakravarty,Vairamuthu T,No,No,NaN,2023-04-15 00:00:00,2023-05-31 00:00:00,AB2P,P2P (Billable to Billable- within Acct in Curr...,NaN,NaN,NaN,NaN,Karthik Manthri,4360,Python,Google Cloud Platform,0.0,0.0,0.0,0.0,80.0,100.0,184.0,100.0,176.0,100.0,168.0,100.0,184.0,100.0,October 2023,April 2023,April 2023,April 2023,BU2-Suganthi,None,DE,1.0
228,229,2023-03-24,NaN,R-0010196-01,Madhava Krishnan C V,US,C00131,PEPSICO APAC,PEPS,PEPSI,P1,DP00057,PEPSICO_GBS_Media Measurement Modeling,Managed Program - FP,DFL-00057-02,Yes,New,Fulfilled,2023-10-04,2023-04-10,2023-06-30,2023-04-15,Technology,Data Science,Data Science,NaN,L1,IC,Internal,Yes,Billable,2023-04-10,2023-06-30,2023-05-09,40,LL,Arvind Mahishi R,Gaurav Kanda,Bhanu Prasad Mahanti,NaN,No,No,NaN,2023-04-10 00:00:00,2023-06-30 00:00:00,B2P,NaN,NaN,NaN,NaN,NaN,Jyotsna Jaykumar Jain,1746,Modeling Fundamentals & Linear Regression,NaN,0.0,0.0,0.0,0.0,120.0,100.0,184.0,100.0,176.0,100.0,0.0,0.0,0.0,0.0,October 2023,April 2023,April 2023,May 2023,FL2-Madha

## Pivot table

In [42]:
input_ops_planned_date = 'January 2000'

In [59]:
pivot_table_input_month_fullfiled_DU_univ = demand_data[(demand_data.Reporting_status==month_fulfilled) | (demand_data.Reporting_status=='D Universe')]

#excluding Jan 2000'January 2000'
pivot_table_input_month_fullfiled_DU_univ_without_Jan00 = demand_data[(demand_data.Reporting_status==month_fulfilled) | (demand_data.Reporting_status=='D Universe') & (demand_data['Ops Planned Date']!=input_ops_planned_date)]

In [60]:
pivot_table_input_month_fullfiled_DU_univ.shape, pivot_table_input_month_fullfiled_DU_univ_without_Jan00.shape

((233, 77), (229, 77))

In [61]:
pivot_demand_universe = pivot_table_input_month_fullfiled_DU_univ.pivot_table(index=['BUOps','Customer Name'] ,values='FTE', columns='Stream1', aggfunc='sum')
pivot_demand_universe = pivot_demand_universe.assign(DU_Total=pivot_demand_universe.sum(axis=1)).reset_index()

In [62]:
pivot_demand_universe.head()

Stream1,BUOps,Customer Name,AC,APP & Design,DE,DS,DevOps,MLE,Prod Support,TC,DU_Total
0,BU1-Muthu,BAYADA,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,1.5
1,BU1-Muthu,CARRIER,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
2,BU1-Muthu,CASHREWARDS,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0
3,BU1-Muthu,COLGATE_GLOBAL,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0
4,BU1-Muthu,COLONIAL ENERGY INC,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [63]:
pivot_demand_universe.shape

(41, 11)

In [64]:
pivot_demand_universe_with_ops_p_d = pivot_table_input_month_fullfiled_DU_univ_without_Jan00.pivot_table(index=['BUOps','Customer Name'] ,values='FTE', columns='Stream1', aggfunc='sum')
pivot_demand_universe_with_ops_p_d = pivot_demand_universe_with_ops_p_d.assign(DU_Ops_Total=pivot_demand_universe_with_ops_p_d.sum(axis=1)).reset_index().reset_index()

In [65]:
pivot_demand_universe_with_ops_p_d.head(2)

Stream1,index,BUOps,Customer Name,AC,APP & Design,DE,DS,DevOps,MLE,Prod Support,TC,DU_Ops_Total
0,0,BU1-Muthu,BAYADA,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,1.5
1,1,BU1-Muthu,CARRIER,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [66]:
pivot_demand_universe_with_ops_p_d.shape

(41, 12)

In [67]:
pivot_demand_universe_with_ops_p_d.drop(['index'], axis=1, inplace=True)

In [68]:
pivot_demand_universe_with_ops_p_d.head()

Stream1,BUOps,Customer Name,AC,APP & Design,DE,DS,DevOps,MLE,Prod Support,TC,DU_Ops_Total
0,BU1-Muthu,BAYADA,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,1.5
1,BU1-Muthu,CARRIER,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
2,BU1-Muthu,CASHREWARDS,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0
3,BU1-Muthu,COLGATE_GLOBAL,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0
4,BU1-Muthu,COLONIAL ENERGY INC,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [43]:
# this is multiindexing here (if anyone want multiindexing, then he can use below code)

# pivot_demand_universe.columns = pd.MultiIndex.from_tuples([('Demand Universe', col) for col in pivot_demand_universe.columns])
# pivot_demand_universe_with_ops_p_d.columns = pd.MultiIndex.from_tuples([('DEMAND UNIVERSE with OPS PLAN DATE', col) for col in pivot_demand_universe_with_ops_p_d.columns])


In [73]:
merged_pivot = pd.merge(pivot_demand_universe, pivot_demand_universe_with_ops_p_d, on=['BUOps', 'Customer Name'], suffixes=('_DU', "_DU_with_OPD"))

In [45]:
# merged_pivot = pd.concat([pivot_demand_universe, pivot_demand_universe_with_ops_p_d], axis=1, ignore_index=False)

In [74]:
merged_pivot.head()

Stream1,BUOps,Customer Name,AC_DU,APP & Design_DU,DE_DU,DS_DU,DevOps_DU,MLE_DU,Prod Support_DU,TC_DU,DU_Total,AC_DU_with_OPD,APP & Design_DU_with_OPD,DE_DU_with_OPD,DS_DU_with_OPD,DevOps_DU_with_OPD,MLE_DU_with_OPD,Prod Support_DU_with_OPD,TC_DU_with_OPD,DU_Ops_Total
0,BU1-Muthu,BAYADA,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,1.5
1,BU1-Muthu,CARRIER,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
2,BU1-Muthu,CASHREWARDS,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0
3,BU1-Muthu,COLGATE_GLOBAL,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0
4,BU1-Muthu,COLONIAL ENERGY INC,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [47]:
# result_pivot.to_excel('aa.xlsx')

In [83]:
# prior fulfilled , input month bsd
prior_fulfulled_input_month_bsd = demand_data[demand_data.Reporting_status==f'Prior Fulfilled, {month[:3]} BSD']
prior_fulfulled_input_month_bsd = prior_fulfulled_input_month_bsd.groupby('Customer Name', as_index=False)['FTE'].sum()
prior_fulfulled_input_month_bsd.rename({'FTE':f'{month[:3]} BSD'}, inplace=True, axis=1)

In [84]:
prior_fulfulled_input_month_bsd.head()

,Customer Name,Aug BSD
0,AMPLIFY HEALTH,3
1,BAYADA,1
2,CARRIER,1
3,CASHREWARDS,1
4,COCHLEAR,2


In [86]:
result = pd.merge(merged_pivot, prior_fulfulled_input_month_bsd, on="Customer Name", how="left")

In [88]:
result.shape

(41, 21)

In [ ]:
# Saving the file
result.to_excel('./result/automat.xlsx') #(you can change the name and location from here)